In [30]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import MinMaxScaler

In [31]:
df = pd.read_csv("fibonacci_sequence.csv")
print(df.head())

   Iteration Value
0          0     0
1          1     1
2          2     1
3          3     2
4          4     3


In [36]:
# Tomemos solo primeros 50 términos para evitar overflow
values = df["Value"].astype(float).values[:50]

# Normalizamos desde el inicio
values = values / np.max(values)

In [ ]:
X, y = [], []
window_size = 5 #how many you want to see in the backwards
for i in range(len(values) - window_size):
    X.append(values[i:i+window_size])
    y.append(values[i+window_size])
    

X = np.array(X).reshape(-1, window_size, 1)
y = np.array(y).reshape(-1, 1)


In [34]:
print(X[:3])
print(y[:3])

[array([0., 1., 1., 2., 3.]), array([1., 1., 2., 3., 5.]), array([1., 2., 3., 5., 8.])]
[5.0, 8.0, 13.0]


In [43]:
#You need to make now reshape the keras, so it has (labels, window_size, characteristics)

"""
X
n_muestras → cuántas secuencias tienes (por ejemplo, 50 ventanas distintas).

pasos_de_tiempo → cuántos elementos hay en cada secuencia (en tu caso, 5 valores Fibonacci por ventana).

características → cuántos valores distintos hay en cada paso (aquí solo 1, el valor mismo).
"""
X = X.astype(np.float32)
y = y.astype(np.float32)

In [44]:
optimizer = keras.optimizers.AdamW(learning_rate=1e-5, clipnorm=1.0)

model = keras.Sequential([
    keras.layers.SimpleRNN(50, activation="tanh", input_shape=(X.shape[1], X.shape[2])),
    keras.layers.Dense(1)
])

#Mean Squared Error (y - yhat)^2
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])

In [45]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_4 (SimpleRNN)        │ (None, 50)             │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,651 (10.36 KB)

 Trainable params: 2,651 (10.36 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=50)

model.compile()

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3m